In [ ]:
!pip install pyspark

In [6]:
import mlflow
mlflow.set_tracking_uri("http://model-repository.stg.dreamplug.net/")
mlflow.set_experiment("spark-test")

<Experiment: artifact_location='s3://dp-batchplatform-stage/ds/model-repository/20', experiment_id='20', lifecycle_stage='active', name='spark-test', tags={}>

In [ ]:
import mlflow.spark
import os
import shutil
from pyspark.sql import SparkSession

spark = (SparkSession.builder
            .config("spark.jars.packages", "org.mlflow:mlflow-spark:1.11.0")
            .master("local[*]")
            .getOrCreate())
df = spark.createDataFrame([
        (4, "spark i j k"),
        (5, "l m n"),
        (6, "spark hadoop spark"),
        (7, "apache hadoop")], ["id", "text"])
import tempfile
tempdir = tempfile.mkdtemp()
df.write.csv(os.path.join(tempdir, "my-data-path"), header=True)
# Enable Spark datasource autologging.
mlflow.spark.autolog()
loaded_df = spark.read.csv(os.path.join(tempdir, "my-data-path"),
                header=True, inferSchema=True)
# Call toPandas() to trigger a read of the Spark datasource. Datasource info
# (path and format) is logged to the current active run, or the
# next-created MLflow run if no run is currently active
with mlflow.start_run() as active_run:
    pandas_df = loaded_df.toPandas()

In [5]:
import mlflow.spark
import os
import shutil
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer
spark = (SparkSession.builder
            .config("spark.jars.packages", "org.mlflow:mlflow-spark")
            .master("local[*]")
            .getOrCreate())
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0) ], ["id", "text", "label"])
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])
model = pipeline.fit(training)
with mlflow.start_run() as run:
    mlflow.spark.log_model(model, "spark-model")